Buiding and training a model to give sentiment of a movie review based on pretrained model ofgoogle, which is trained with google news dataset.

In [3]:
import tensorflow as tf
#tf.__version__
import tensorflow_datasets as tfds

Listing all the dataset using tfds module and list_builder function. 

In [ ]:
tfds.list_builders() #'imdb_reviews'

Load the imdb review dataset

In [12]:
(train_data,validation_data),test_data=tfds.load(name='imdb_reviews',split=(['train[:70%]', 'train[70%:]'],tfds.Split.TEST),as_supervised=True)

To see the data we have iterate through batches

In [21]:
train_data_batch,train_label_batch=next(iter(train_data.batch(5)))
print(train_data_batch,train_label_batch)

tf.Tensor(
[b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
 b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot de

In [22]:
!pip install tensorflow_hub

Import hub module for using the advantage of transfer learning from google's pretrained model with google news dataset.

In [23]:
import tensorflow_hub as hub

In [26]:
pretrained_model='https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1' #embedding dim=20
hub_layer=hub.KerasLayer(pretrained_model,input_shape=[], dtype=tf.string,trainable=True)

Test our hub layer on train_data_batch, we see the two reviews from train_data_batch is converted into vectors of size 20

In [29]:
hub_layer(train_data_batch[:2])


<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ]],
      dtype=float32)>

Build model

In [32]:
from tensorflow.keras import layers
from keras.layers import Dense

In [33]:
model=tf.keras.Sequential()
model.add(hub_layer)# adding hub at the begging of model from which the next dense layer will take input . hub layer has 20 neurons as the vector size
model.add(Dense(10,activation='relu'))#next layer has 10
model.add(Dense(10,activation='relu'))
model.add(Dense(1,activation='sigmoid'))# output layer

In [34]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 10)                210       
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 400,351
Trainable params: 400,351
Non-trainable params: 0
_________________________________________________________________


In [41]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [43]:
model.fit(train_data.shuffle(1000).batch(128),validation_data=validation_data.shuffle(1000).batch(128),epochs=20)

Epoch 1/20
137/137 [==============================] - 5s 32ms/step - loss: 0.2799 - accuracy: 0.8937 - val_loss: 0.3479 - val_accuracy: 0.8571
Epoch 2/20
137/137 [==============================] - 5s 33ms/step - loss: 0.2344 - accuracy: 0.9144 - val_loss: 0.3341 - val_accuracy: 0.8629
Epoch 3/20
137/137 [==============================] - 5s 33ms/step - loss: 0.1955 - accuracy: 0.9315 - val_loss: 0.3337 - val_accuracy: 0.8647
Epoch 4/20
137/137 [==============================] - 5s 33ms/step - loss: 0.1643 - accuracy: 0.9452 - val_loss: 0.3410 - val_accuracy: 0.8688
Epoch 5/20
137/137 [==============================] - 5s 33ms/step - loss: 0.1375 - accuracy: 0.9563 - val_loss: 0.3574 - val_accuracy: 0.8659
Epoch 6/20
137/137 [==============================] - 5s 33ms/step - loss: 0.1151 - accuracy: 0.9671 - val_loss: 0.3700 - val_accuracy: 0.8676
Epoch 7/20
137/137 [==============================] - 5s 33ms/step - loss: 0.0963 - accuracy: 0.9749 - val_loss: 0.3969 - val_accuracy: 0.8669

In [51]:
test_data_batch,test_data_label=next(iter(test_data.batch(2)))
print(test_data_batch)
print(test_data_label)

tf.Tensor(
[b"There are films that make careers. For George Romero, it was NIGHT OF THE LIVING DEAD; for Kevin Smith, CLERKS; for Robert Rodriguez, EL MARIACHI. Add to that list Onur Tukel's absolutely amazing DING-A-LING-LESS. Flawless film-making, and as assured and as professional as any of the aforementioned movies. I haven't laughed this hard since I saw THE FULL MONTY. (And, even then, I don't think I laughed quite this hard... So to speak.) Tukel's talent is considerable: DING-A-LING-LESS is so chock full of double entendres that one would have to sit down with a copy of this script and do a line-by-line examination of it to fully appreciate the, uh, breadth and width of it. Every shot is beautifully composed (a clear sign of a sure-handed director), and the performances all around are solid (there's none of the over-the-top scenery chewing one might've expected from a film like this). DING-A-LING-LESS is a film whose time has come."
 b"A blackly comic tale of a down-trodden pri

In [52]:
model.predict(test_data_batch)

array([[0.99999213],
       [0.9775069 ]], dtype=float32)

Another pretrained model with 128 dimension of word embedding in cvector space

In [36]:
hub2=hub.KerasLayer('https://tfhub.dev/google/nnlm-en-dim128/2') #embedding dimension is 128

In [38]:
hub2().shape

TensorShape([3, 128])